In [1]:
import pandas as pd
from binance.client import Client
from Binance_api import api_key, secret_key
from datetime import datetime
from datetime import timedelta
import time
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
import plot_utils 
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

warnings.filterwarnings('ignore')

client = Client(api_key, secret_key,tld='us')
rsi = 0
bottoms_5 = [9999999,99999999,99999999,99999999,9999999]
rsi_l = [9999999,99999999,99999999,99999999,9999999]

ModuleNotFoundError: No module named 'Binance_api'

In [3]:
df = pd.read_csv('DOGE_15min_10AUG.csv')

In [172]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band

symbols = ['DOGE']

nstd = 3
def rsi(symbol, i):
    if df[f'{symbol}_rsi'].iloc[i] > rsi:
        rsi = df[f'{symbol}_rsi'].iloc[i]
    return rsi

def bottoms(new_bottom,rsi_new):
    is_new = 0
    for a in range(len(bottoms)):
        if new_bottom < bottoms[a]:
            is_new = 1
    if is_new == 1:
        br = len(bottoms_5) - 1
        while br > 0:
            bottoms_5[br] = bottoms_5[br-1]
            rsi_l[br] = rsi_l[br-1]
            br -= 1
        br[0] = new_bottom
        rsi_l = rsi_new
     
def divergence():
    low_n = 0
    for a in range(len(bottoms)):
        if bottoms[0] < bottoms[a] and rsi_l[0] > rsi_l[a]:
            low_n =+ 1 
    if low_n == 4:
        return True


    



for symbol in symbols:
    df[f'{symbol}_sma'] = sma(df[f'{symbol}-USD_Open'],20)
    df[f'{symbol}_upper_band'], df[f'{symbol}_lower_band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_sma'], 20, nstd)        
    df[f'{symbol}_rsi'] = ta.RSI(df[f'{symbol}-USD_Close'])

        
df.dropna(inplace=True)

In [173]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = trading_fee_multiplier
        
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
        
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append( [self.balance_unit, time, sell_price] )
        self.balance_unit = 'USDT'

# VIP level 0, paying fees with BNB = 0.075%

        

In [174]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=1) #0.99925

In [175]:
last_buy = 0
for i in range(len(df)):
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
                bottoms(df[f'{symbol}-USD_Low'].iloc[i], df[f'{symbol}_lower_band'].iloc[i])
                
                

            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] < 25: #buy signal
                env.buy(symbol, df[f'{symbol}_lower_band'].iloc[i], df['OpenTime'].iloc[i])
                last_buy = df[f'{symbol}_lower_band'].iloc[i]
                break

    if env.balance_unit != 'USDT':
        if last_buy*0.98 > df[f'{env.balance_unit}-USD_Close'].iloc[i]:
            env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i], df['OpenTime'].iloc[i])
        elif df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] > 60: #sell signal
            env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i], df['OpenTime'].iloc[i])

if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])

In [176]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print('starting balance: 100 USDT')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')


num buys: 358
num sells: 358
starting balance: 100 USDT
ending balance: 2419.108833817974 USDT


In [178]:
balance = 100
profit_total = 0
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(buys)
    print(sells)
    print('P/L = 'f'{profit}')
    balance += balance*(profit/100)
    print(balance)
    print('\n')
    

['BTC', '2021-11-11 06:00:00', 66274.1634415098]
['BTC', '2021-11-11 07:20:00', 68032.95327984204]
P/L = 2.653809187473888
102.65380918747388


['BTC', '2021-11-14 04:30:00', 64285.35946116092]
['BTC', '2021-11-14 06:55:00', 64341.42048864935]
P/L = 0.08720652409559548
102.743330006318


['BTC', '2021-11-14 18:20:00', 64341.40711287644]
['BTC', '2021-11-14 22:40:00', 64560.28329532079]
P/L = 0.3401793530259045
103.09284160161077


['BTC', '2021-11-16 00:20:00', 64974.97638885021]
['BTC', '2021-11-16 08:10:00', 64114.24831457299]
P/L = -1.3247070212478285
101.72716349051034


['BTC', '2021-11-16 10:05:00', 63170.71630306972]
['BTC', '2021-11-16 11:00:00', 64644.50429184492]
P/L = 2.333024026044783
104.1004826557578


['BTC', '2021-11-16 13:25:00', 61162.866780277385]
['BTC', '2021-11-16 20:00:00', 61406.393650306745]
P/L = 0.3981613074223844
104.51497049853297


['BTC', '2021-11-16 20:05:00', 59740.867273794516]
['BTC', '2021-11-17 00:45:00', 61371.91948827856]
P/L = 2.730211811303085
1

In [34]:
a = [1,6,5,3]
x = 8

b = len(a) - 1
while b > 0:
    a[b] = a[b-1]
    b -= 1
a[0] = x


    


In [35]:
a

[8, 1, 6, 5]